In [1]:
!pip install sparknlp pymongo

Defaulting to user installation because normal site-packages is not writeable


In [23]:
import os
from pprint import pprint
import pymongo
from pymongo import MongoClient    
import pyspark
from pyspark.ml import PipelineModel
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
import time
import re
from zipfile import ZipFile

# Writing to DB

In [9]:
conf = pyspark.SparkConf()
conf.set('spark.jars.packages', 
         "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,com.databricks:spark-xml_2.12:0.18.0,com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3")
conf.set('spark.driver.memory','8g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)
spark

:: loading settings :: url = jar:file:/ext3/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dmb443/.ivy2/cache
The jars for the packages stored in: /home/dmb443/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
com.databricks#spark-xml_2.12 added as a dependency
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-09e7966a-b777-4604-b624-19b7f3e067af;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
	found com.databricks#spark-xml_2.12;0.18.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.type

24/04/29 20:24:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [15]:
# user = "ob2205"
# PATH = '/scratch/ob2205/shared/ip/ip_address.txt'
# ip_df = spark.read.text(PATH)
# uri = ip_df.head(1)[0].value
uri = "mongodb+srv://dmb443:<password>@bigdatafinalproject.sl03s.mongodb.net/?retryWrites=true&w=majority&appName=BigDataFinalProject"

In [14]:
uri

'10.32.32.5'

In [16]:
client = MongoClient(uri)

In [32]:
client.server_info()

{'version': '7.0.8',
 'gitVersion': 'c5d33e55ba38d98e2f48765ec4e55338d67a4a64',
 'modules': ['enterprise'],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [7, 0, 8, 0],
 'openssl': {'running': 'OpenSSL 1.0.2k-fips  26 Jan 2017',
  'compiled': 'OpenSSL 1.0.2k-fips  26 Jan 2017'},
 'buildEnvironment': {'distmod': 'amazon2',
  'distarch': 'aarch64',
  'cc': '/opt/mongodbtoolchain/v4/bin/gcc: gcc (GCC) 11.3.0',
  'ccflags': '-Werror -include mongo/platform/basic.h -ffp-contract=off -fasynchronous-unwind-tables -g2 -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -gdwarf-5 -fno-omit-frame-pointer -fno-strict-aliasing -O2 -march=armv8.2-a -mtune=generic -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -gdwarf64 -Wa,--nocompress-debug-sections -Wimplicit-fallthrough=5',
  'cxx': '/opt/mongodbtoolchain/v4/bin

In [34]:
db=client.bigdata

In [8]:
db

Database(MongoClient(host=['bigdatafinalproject-shard-00-02.sl03s.mongodb.net:27017', 'bigdatafinalproject-shard-00-00.sl03s.mongodb.net:27017', 'bigdatafinalproject-shard-00-01.sl03s.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='BigDataFinalProject', authsource='admin', replicaset='atlas-qlbwa4-shard-0', tls=True), 'bigdata')

In [11]:
# Extract articles from zip

# source_path = "/scratch/work/public/proquest/proquest_hnp/BostonGlobe/BG_20151210212722_00001.zip"
# with ZipFile(source_path, "r") as zip:
#     zip.extractall('zip_tmp')

In [9]:
df = spark.read\
    .option('rootTag', 'Record')\
    .option('rowTag', 'Record')\
    .option('recursiveFileLookup', 'true')\
    .format("xml").load("zip_tmp")

In [18]:
# Start SparkNLP
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

In [ ]:
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

In [19]:
class YakePipeline(Pipeline):
    """
    A pipeline for extracting keywords using YAKE.

    Example:
    pipeline = YakePipeline()
    processed_df = pipeline.fit(df).transfrom(df)
    """
    def __init__(self):
        super(YakePipeline, self).__init__()
        self.stopwords = StopWordsCleaner().getStopWords()
        self.document = DocumentAssembler() \
                .setInputCol("FullText") \
                .setOutputCol("document")
        self.sentenceDetector = SentenceDetector() \
                .setInputCols("document") \
                .setOutputCol("sentence")
        self.token = Tokenizer() \
                .setInputCols("sentence") \
                .setOutputCol("token") \
                .setContextChars(["(", ")", "?", "!", ".", ","])
        self.keywords = YakeKeywordExtraction() \
                .setInputCols("token") \
                .setOutputCol("keywords") \
                .setMinNGrams(1) \
                .setMaxNGrams(3)\
                .setNKeywords(20)\
                .setStopWords(self.stopwords)
        self.setStages([self.document, self.sentenceDetector, self.token, self.keywords])


In [13]:
# Extract keywords and append to df
yake_pipeline = YakePipeline()

result = yake_pipeline.fit(df).transform(df)\
    .drop("document")\
    .drop("token")\
    .drop("sentence")

In [16]:
# df now includes keywords, along with other outputs from pipeline
result.printSchema()

root
 |-- Abstract: string (nullable = true)
 |-- ActionCode: string (nullable = true)
 |-- AlphaPubDate: string (nullable = true)
 |-- Contributor: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ContribRole: string (nullable = true)
 |    |    |-- FirstName: string (nullable = true)
 |    |    |-- LastName: string (nullable = true)
 |    |    |-- MiddleName: string (nullable = true)
 |    |    |-- NameSuffix: string (nullable = true)
 |    |    |-- OrganizationName: string (nullable = true)
 |    |    |-- OriginalForm: string (nullable = true)
 |    |    |-- PersonName: string (nullable = true)
 |    |    |-- PersonTitle: string (nullable = true)
 |-- DateTimeStamp: long (nullable = true)
 |-- FullText: string (nullable = true)
 |-- LanguageCode: string (nullable = true)
 |-- NumericPubDate: long (nullable = true)
 |-- ObjectType: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Pagination: string (nullable = true)
 |

In [23]:
# Write to mongo
start = time.time()
result.write.format("com.mongodb.spark.sql.DefaultSource")\
    .mode("overwrite")\
    .option("database", "bigdata")\
    .option("collection", "newspapers")\
    .option("uri", uri)\
    .save()
stop = time.time()
print(stop - start)

158.52524161338806


# Querying


In [210]:
def clean_query(query, stopwords):
    """Takes an user query and stopwords array and returns a keywords array"""
    query = re.sub("[^a-z]", " ", query.strip().lower())
    query = re.sub("  +", " ", query)
    keywords = query.split()
    keywords = [word for word in keywords if word not in stopwords]
    return keywords


def keyword_search(keywords, db, k):
    """Takes keywords array and returns top-k articles with those keywords."""
    # aggregate pipeline to find top-k articles based on input keywords
    data = db.aggregate([
        {"$project":{"keywords.result": 1, "keywords.metadata.score": 1}},
        {"$unwind": {"path": "$keywords", "preserveNullAndEmptyArrays": False}},
        {"$match": {"keywords.result":  {"$in": keywords}}},
        {"$group": { "_id": "$_id", "keywords": {"$addToSet": "$keywords"}}},
        {"$unwind": {"path": "$keywords", "preserveNullAndEmptyArrays": False}},
        {"$addFields": {"keywords.reciprocal": {"$divide": [1, {"$convert": {"input":"$keywords.metadata.score", "to": "double", "onError": 10**7, "onNull": 10**7}}]}}},
        {"$group": { "_id": "$_id", "score": {"$sum": "$keywords.reciprocal"},"keywords": {"$addToSet": "$keywords.result"}}},
        {"$sort": {"score": -1}},
        {"$limit": k}
    ])
    
    # TO DO
    # Optimize pipeline to remove redundant groupBys and/or unwinds
    # Remove duplicates
        
    return data


In [208]:
stopwords = StopWordsCleaner().getStopWords()
print(clean_query("This is a test query for our Historical Newspaper Search Engine!", stopwords))

['test', 'query', 'historical', 'newspaper', 'search', 'engine']


In [211]:
output = keyword_search(["china", "green"], db.newspapers, 5)

for doc in output:
    pprint(doc)

{'_id': ObjectId('662ec7794eb4c37be22c4a1b'),
 'keywords': ['china'],
 'score': 47.62385286532919}
{'_id': ObjectId('662ec77d4eb4c37be22c4d9a'),
 'keywords': ['green'],
 'score': 31.19757243596742}
{'_id': ObjectId('662ec7584eb4c37be22c2a8b'),
 'keywords': ['china', 'green'],
 'score': 31.137394526687103}
{'_id': ObjectId('662ec7554eb4c37be22c26cb'),
 'keywords': ['green'],
 'score': 29.31299241197076}
{'_id': ObjectId('662ec75f4eb4c37be22c30e2'),
 'keywords': ['china'],
 'score': 26.016442094267518}


# Visualizations

In [187]:
%%html
<style>
.box_style{
    width:100%;
    border : None;
    height: auto;
    background-color:#EEE;
    color=white;
}
.side_bar{
    width:100%;
    border: None;
    height: auto;
    background-color:#66b2b2;
    color=white;
}

.widget-label {
    color: white !important;
}

.widget_text {
    border-radius: 8px;
}
.button_style {
    margin-top: 15px;
}
</style>

In [188]:
import ipywidgets as ipw
from IPython.display import HTML, display, clear_output, Javascript
from bson.objectid import ObjectId
import re

def view_doc_clicked(b):
    _id = b.tooltip
    if len(_id):
        doc = db.newspapers.find_one({"_id": ObjectId(_id)})
        if doc:
            show_document(doc['RecordTitle'], doc['FullText'])
    

def show_document(header, text):
    text = re.sub("[^0-9a-zA-Z]", " ", text)
    text = re.sub(" +", " ", text).strip()
    
    display(Javascript("""
        require(
            ["base/js/dialog"], 
            function(dialog) {
                dialog.modal({
                    title: '%s',
                    body: '%s',
                    buttons: {
                        'Done': {}
                    }
                });
            }
        );
        """ % (header, text)))


notify_output = ipw.Output()
display(notify_output)

@notify_output.capture()
def popup(text):
    clear_output()
    display(HTML("<script>alert('{}');</script>".format(text)))

def create_result_option(record_id, record_title, ctx_text, publisher, pub_date, object_id):
    items_layout = ipw.Layout(width='90%')
    
    children = []
    row_layout = ipw.Layout(display='flex', flex_flow='row', align_items='stretch', border_bottom='solid 2px lightgrey', padding='5px')
    sn_box = ipw.HBox([ipw.HTML(f"{record_id}.")], layout=ipw.Layout(width='3%', margin='5px 5px 5px 5px'))
    rn_box = ipw.HBox(children=[], layout=ipw.Layout(width='10%', margin='5px 5px 5px 5px'))
    
    if len(record_title):
        btnView = ipw.Button(description="View", tooltip=str(object_id))
        btnView.on_click(view_doc_clicked)
        rn_box.children = [btnView]
        
        
        children.append(ipw.HTML(f"<b><font color='#1b75d0'; size=3px>{record_title}", layout=items_layout))
    if len(ctx_text):
        children.append(ipw.HTML(ctx_text, layout=items_layout))
    if len(publisher):
        children.append(ipw.HTML(f"<font color='grey'><b>Publisher:</b> {publisher}", layout=items_layout))
    if len(pub_date):
        children.append(ipw.HTML(f"<font color='grey'><b>Date:</b> {pub_date}", layout=items_layout))
    
    row_content = ipw.VBox(children = children, layout=ipw.Layout(width='90%'))
    row = ipw.HBox(children=[sn_box, row_content, rn_box], layout=row_layout)
 
    return row

Output()

In [189]:
def search_keyword(keyword):
    data = db.newspapers.aggregate([             
        {
            '$facet': {
                'docs': [
                    { '$unwind': '$keywords' },
                    { '$match': {'keywords.result': keyword} }
                ],
                'count': [{'$count': "count"} ]
            }
        },
        {
            '$addFields': { 'count': { '$arrayElemAt': ["$count.count", 0] } }
        }
    ])
    
    
    # ToDo: Limit to say 5 results
    data = data.next()
    i = 1
    children = []
    if len(data['docs']) < 1:
        children.append(create_result_option("#", '', "No Newspaper found with this keyword", "", '', ''))
        return children

    for d in data['docs']:
        # ctx_txt: take keyword with highest score and contexualize it
        start = d["keywords"]['begin'] or 0
        end = d["keywords"]['end'] or 0
        ctx_text=word_highlighter(d["FullText"], start, end, 90)

        children.append(create_result_option(i, d["RecordTitle"], ctx_text, d["Publisher"], d["AlphaPubDate"], d['_id']))
        i += 1
    return children   

def word_highlighter(fulltext, start, end, ctx_length=100):
    tot_length = len(fulltext)
    chunk_start = 0 if start-ctx_length < 0 else start-ctx_length
    chunk_end = tot_length if end+ctx_length > tot_length else end+ctx_length
    return fulltext[chunk_start:start] + \
        f"<span style='background-color: yellow;'>{fulltext[start:end + 1]}</span>" + \
        fulltext[end + 1:chunk_end]

# search_keyword("grass")

In [190]:
hor_layout = ipw.Layout(align_content='stretch', margin='0.1% 1% 0.1% 2% ', width='100%')

app_title = ipw.HTML('<h1> YouTube for Newspapers </h1>', layout=hor_layout)
app_footer = ipw.HTML('<p> Apache Spark, HDFS, MongoDB, NLP</p>', layout=hor_layout)

headerBox = ipw.HBox([app_title]).add_class('box_style')
footerBox = ipw.HBox([app_footer]).add_class('box_style')

txt_keyword = ipw.Text(placeholder='Enter a Keyword', description='Keyword:', disabled=False, layout=ipw.Layout(width='auto', 
    margin='15px 10px 5px 2px')).add_class('widget_text')
btnSearch = ipw.Button(description="Search!", icon='search').add_class('button_style')
box_layout = ipw.Layout(display='flex', flex_flow='column', align_items='center', width='100%')
btnContainer = ipw.HBox(children=[btnSearch], layout=box_layout)


def on_button_clicked(b):
    if len(txt_keyword.value.strip()) < 1:
        popup("Enter search Key.")
    else:
        main_panel.children = search_keyword(txt_keyword.value.lower())

btnSearch.on_click(on_button_clicked)

side_bar = ipw.VBox([txt_keyword, btnContainer]).add_class('side_bar')
main_panel = ipw.VBox([])

ipw.AppLayout(header=headerBox, left_sidebar=side_bar, center=main_panel, right_sidebar=None, footer=footerBox,
    pane_widths=[2, 7, 0],
    pane_heights=[1, 9, '40px'])

AppLayout(children=(HBox(children=(HTML(value='<h1> YouTube for Newspapers </h1>', layout=Layout(align_content…